# Projet maison n° 4

In [8]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas  (par ex. fonction \"concat\" ou méthode \"append\"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation ci-dessus).

In [9]:
# 1) implémentation avec glob + append
import glob

def df_names_us():
    df = pd.DataFrame()
    files = glob.glob(r'C:\Users\julie\OneDrive\Documents\Telecom\MDI721 - Exos\Julien-Muller\session 4\names\*.txt')
    files.sort()
    for filename in files:
        csv = pd.read_csv(filename,
                          names=['name', 'gender', 'births'])
        csv['year'] = int(filename[-8:-4]) # yobAAAA.txt
        df = df.append(csv, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [10]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 14 s


(1989401, 4)

In [11]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [ ]:
# 2) implémentation avec range + concat
def df_names_us():
    dfs = []
    for year in range(1880, 2020):
        csv = pd.read_csv(f'names/yob{year}.txt',
                          names=['name', 'gender', 'births'])
        csv['year'] = year
        dfs.append(csv)
    df = pd.concat(dfs, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [ ]:
%%time
df_us = df_names_us()
df_us.shape

In [ ]:
# head
df_us.head()

**slice vs regex**

In [ ]:
# slice 140 x 96.1ns = ~14ms
%timeit 'yob2020.txt'[-8: -4]

In [ ]:
# regex 140 x 1.37µs = ~192ms (x14)
import re
%timeit re.search('(\d+)', 'yob2020.txt').group(0)

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [ ]:
def df_names_fr():
    # genres
    d = {'1': 'M', '2': 'F'}
    # read_csv
    df = pd.read_csv('nat2019.csv',
                      sep=';',
                      header=0,
                      names=['gender', 'name', 'year', 'births'],
                      converters={
                          'gender': d.get,
                          'name': str.title
                      })
    # clean
    df = df.loc[(df['name'].str.len() > 1)
                & (df['year'] != 'XXXX')
                & ~df['name'].str.startswith('_')]
    # types
    df['year'] = df['year'].astype(int)
    # ordre colonnes
    df = df[['year', 'name', 'gender', 'births']]
    # tri
    df = df.sort_values(['year', 'gender', 'births', 'name'],
                   ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    
    return df

In [ ]:
%%time
df_fr = df_names_fr()
df_fr.shape

In [ ]:
# head
df_fr.head()

In [ ]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

In [ ]:
# impact du converters / name
d = {'1': 'M', '2': 'F'}

df = pd.read_csv('nat2019.csv',
                  sep=';',
                  header=0,
                  names=['gender', 'name', 'year', 'births'],
                  converters={
                      'gender': d.get,
                      #'name': str.title
                  })

df['name'].isnull().sum()

In [1]:
# capitalize
'JEAN-MARIE'.capitalize()

'Jean-marie'

In [2]:
# title
'JEAN-MARIE'.title()

'Jean-Marie'

In [ ]:
# comment faire title avec capitalize :)
'-'.join(map(str.capitalize, 'JEAN-MARIE'.split('-')))

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [ ]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=[0, 1, 3, 4, 5],  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes monnaies
    cols = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=True)
    cols.iloc[0] = 'Date'
    df.columns = cols[0]

    # selection des devises
    df = df[['Date'] + devises]

    # drop na
    df = df.dropna()

    # set index
    df = df.set_index('Date')
    
    return df

In [ ]:
%%time
df_tx = df_taux_change(['CHF', 'GBP', 'USD'])
df_tx.shape

In [ ]:
# head
df_tx

### Tests

In [ ]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [ ]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [ ]:
# run tests

run_tests()

### Un peu de Data Science...

**Question n° 1**

Pourquoi le value_counts() du "gender" donne-t-il un tel écart entre F et M ?

In [ ]:
# value_counts du gender

df_us['gender'].value_counts()

In [ ]:
# pct_change

df_us['gender'].value_counts().pct_change()

**Question n° 2**

Pourquoi le value_counts() du "name" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender

df_us['name'].value_counts().head(16)

**Question n° 3**

Pourquoi le value_counts() du "year" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender

df_us['year'].value_counts().head(16)

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

### Méthodes de reshaping (1)

#### pivot_table

La méthode pivot_table() prend en argument :
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions.

On obtient NaN s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [6]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

,A,B,C
0,1,1,1
1,1,1,2
2,1,2,-1
3,2,1,4
4,2,1,5


In [7]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='mean')

B,1,2
A,,
1,1.5,-1.0
2,4.5,NaN


**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

In [14]:
df_us.pivot_table(values = 'births',
                 index = 'year',
                 columns = 'gender',
                 aggfunc = 'sum')

gender,F,M
year,,
1880,90994,110490
1881,91953,100743
1882,107847,113686
1883,112319,104625
1884,129019,114442
...,...,...
2015,1781725,1913059
2016,1767902,1893471
2017,1721550,1845472


**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

In [15]:
df_us.pivot_table(values = 'name',
                 index = 'year',
                 columns = 'gender',
                 aggfunc = 'nunique')

gender,F,M
year,,
1880,942,1058
1881,938,997
1882,1028,1099
1883,1054,1030
1884,1172,1125
...,...,...
2015,19099,14039
2016,18847,14181
2017,18400,14229


#### crosstab

crosstab() est une fonction de reshaping qui prend 2 colonnes d'un DataFrame en argument et produit le décompte croisé des occurrence.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [16]:
# exemple
pd.crosstab(df['A'], df['B'])

B,1,2
A,,
1,2,1
2,2,0


In [17]:
# initial
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

,year,name,gender,births,initial,terminal
0,1880,Mary,F,7065,M,Y
1,1880,Anna,F,2604,A,A
2,1880,Emma,F,2003,E,A
3,1880,Elizabeth,F,1939,E,H
4,1880,Minnie,F,1746,M,E


In [18]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'])

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,63865,370,717,3140,30176,504,205,11110,8124,297,...,68,6401,9744,2371,646,304,420,594,8720,753
B,12801,236,150,2986,15742,44,179,2425,2290,76,...,0,2532,1989,3801,336,52,236,272,8992,316
C,29034,418,372,2102,31177,201,789,1856,3694,63,...,0,5623,6964,2206,165,6,33,70,11772,743
D,31730,43,1070,3062,26176,43,362,3700,4471,99,...,26,2130,8156,1390,186,151,414,171,8553,377
E,22847,143,495,2465,17586,48,265,4229,1990,51,...,29,4618,2817,3395,450,13,54,169,4444,126
F,8316,0,284,1494,5597,0,190,938,468,6,...,33,1211,2091,789,57,0,1,320,2117,673
G,10542,62,18,2538,11982,196,502,995,1810,23,...,0,2227,2394,1921,18,197,0,2,4529,22
H,5921,215,19,2643,6443,95,488,2450,2165,7,...,11,2420,1765,2284,185,7,112,75,5825,120
I,10442,4,745,403,3550,14,226,1681,1027,9,...,52,864,1628,199,200,2,0,0,1695,301


In [21]:
# pivot_table
df_us.pivot_table(values='name',
                  index='initial',
                  columns='terminal',
                  aggfunc='nunique').fillna(0).astype(int)

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,3551,33,28,152,1458,26,19,905,569,22,...,14,321,480,141,56,30,18,16,428,64
B,718,7,9,74,611,3,14,153,147,5,...,0,117,75,99,20,2,9,20,329,20
C,1545,22,21,57,1230,6,29,170,201,1,...,0,195,233,71,22,2,4,5,368,24
D,1737,5,28,120,1192,2,17,305,323,10,...,1,132,356,59,11,5,6,13,391,33
E,920,9,21,78,704,3,8,250,152,3,...,1,159,142,94,15,2,7,6,203,17
F,376,0,8,39,190,0,5,48,35,1,...,5,52,69,22,7,0,1,8,78,23
G,452,7,4,86,451,7,17,54,106,6,...,0,82,111,53,4,6,0,2,148,8
H,289,12,6,75,296,5,21,147,128,3,...,3,104,71,70,20,2,1,7,178,8
I,539,1,21,20,176,3,4,108,93,2,...,5,49,84,23,16,1,0,0,74,10


In [ ]:
# Z x Z
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

In [ ]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]